In [20]:
import os
import sys
from time import perf_counter
import cupy as cp
__file__="."
test_dir = os.path.dirname(os.path.abspath(__file__))
sys.path.insert(0, os.path.join(test_dir, ".."))
from pyquda import init, core, quda, mpi
import pyquda.pyqcu as pyquda_pyqcu
from pyquda.enum_quda import QudaParity
from pyquda.field import LatticeFermion
from pyquda.utils import gauge_utils
import pyqcu

In [5]:
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [32, 32, 32, 64]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx // Gx, Ly // Gy, Lz // Gz, Lt // Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)

In [6]:
round=1
# generate a vector p randomly
p = LatticeFermion(latt_size, cp.random.randn(Lt, Lz, Ly, Lx, Ns, Nc * 2).view(cp.complex128))
Mp = LatticeFermion(latt_size)
# Mp1 = LatticeFermion(latt_size)
# Mp2 = LatticeFermion(latt_size)
print('===============round ', round, '======================')
# Set parameters in Dslash and use m=-3.5 to make kappa=1
dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
# Generate gauge and then load it
U = gauge_utils.gaussGauge(latt_size, round)
dslash.loadGauge(U)
cp.cuda.runtime.deviceSynchronize()
t1 = perf_counter()
quda.dslashQuda(Mp.even_ptr, p.odd_ptr, dslash.invert_param, QudaParity.QUDA_EVEN_PARITY)
quda.dslashQuda(Mp.odd_ptr, p.even_ptr, dslash.invert_param, QudaParity.QUDA_ODD_PARITY)
cp.cuda.runtime.deviceSynchronize()
t2 = perf_counter()
print(f'Quda dslash: {t2 - t1} sec')
# # then execute my code
# param = pyquda_pyqcu.QcuParam()
# param.lattice_size = latt_size
# grid = pyquda_pyqcu.QcuParam()
# grid.lattice_size = grid_size
# cp.cuda.runtime.deviceSynchronize()
# t1 = perf_counter()
# pyquda_pyqcu.mpiDslashQcu(Mp1.even_ptr, p.odd_ptr, U.data_ptr, param, 0, grid)
# pyquda_pyqcu.mpiDslashQcu(Mp1.odd_ptr, p.even_ptr, U.data_ptr, param, 1, grid)
# cp.cuda.runtime.deviceSynchronize()
# t2 = perf_counter()
# print(f'QCU dslash: {t2 - t1} sec')
# print('quda difference: ', cp.linalg.norm(Mp1.data - Mp.data) / cp.linalg.norm(Mp.data))

===============round  1 ======================
Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01
Quda dslash: 0.20183226200242643 sec


In [10]:
pyqcu_param={'Lx':Lx, 'Ly':Ly, 'Lz':Lz, 'Lt':Lt}

In [23]:
print(type(Mp.even.data))

<class 'cupy.cuda.memory.MemoryPointer'>


In [22]:
pyqcu.wilson_dslash.run(pyqcu_param, p.even)